In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andyczhao/covidx-cxr2

In [ ]:
!unzip covidx-cxr2.zip

In [ ]:
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,Layer,ReLU, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from skimage.transform import radon, rescale
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage import feature
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D,LayerNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from tensorflow.keras.layers import Layer 

In [ ]:
import os
import glob
train_files = []
train_dir = "/content/train/"
dir_png = os.path.join(train_dir,"*.png")
dir_jpg = os.path.join(train_dir,"*.jpg")
dir_jpeg = os.path.join(train_dir,"*.jpeg")
png_files = glob.glob(dir_png)
jpg_files = glob.glob(dir_jpg)
jpeg_files = glob.glob(dir_jpeg)
train_files.extend(png_files)
train_files.extend(jpg_files)
train_files.extend(jpeg_files)
train_files.sort()


In [ ]:
len(train_files)

In [ ]:
dic = {}
for j in range(len(train_files)):
  b = train_files[j].split("/")[-1]
  dic[b] = train_files[j]


In [ ]:
with open('/content/train.txt') as f:
    lines = f.readlines()

titles = []
for i in range(len(lines)):
  a = lines[i].split(" ")
  titles.append(a[2])



In [ ]:
set(titles)

In [ ]:
with open('/content/train.txt') as f:
    lines = f.readlines()
covid_train = []
Noncovid_train = []
for i in range(len(lines)):
  a = lines[i].split(" ")
  if a[2] == "positive":
      try:
          covid_train.append(dic[a[1]])
      except:
          print("oops")  
      
  else:
      try:
          Noncovid_train.append(dic[a[1]])
      except:
          print(a[1]) 

covid_train.sort()
Noncovid_train.sort()

In [ ]:
print(len(covid_train),len(Noncovid_train))

In [ ]:
train_dic = {}
for f in covid_train[:14841]:
  train_dic[f] = [1,0]
for f in Noncovid_train[:12591]:
  train_dic[f] = [0,1]

In [ ]:
val_dic = {}
for f in covid_train[14841:]:
  val_dic[f] = [1,0]
for f in Noncovid_train[12591:]:
  val_dic[f] = [0,1]

In [ ]:
import random
l_val = list(val_dic.items())
random.Random(4).shuffle(l_val)

In [ ]:
import random
l = list(train_dic.items())
random.Random(4).shuffle(l)

In [ ]:
len(l)

In [ ]:
def parse_image(l_i):
  filename,label = l_i
  image = tf.io.read_file(filename)
  image = tf.image.decode_image(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [224, 224])
#   image = tf.concat([image,image,image],axis = -1)
  return image, label

In [ ]:
import numpy as np
import keras

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_dic, batch_size):
        'Initialization'

        self.batch_size = batch_size
        self.list_dic = list_dic
   
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_dic) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        list_dic_temp = []
        for k in range(index*self.batch_size,(index+1)*self.batch_size):
            list_dic_temp.append(self.list_dic[k])
        X, y = self.__data_generation(list_dic_temp)
        return X, y

    def __data_generation(self, list_dic):
        'Generates data containing batch_size samples'

        x = []
        y = []

        for filename,label in (list_dic):
     
            img = cv2.imread(filename)
            img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
            img = img.astype('float32')/255.0
            x.append(img)
            y.append(label)
        return np.asarray(x), np.asarray(y)

In [ ]:
l_dataset = DataGenerator(l,16)

In [ ]:
x,y = l_dataset.__getitem__(0)

In [ ]:
y.shape

In [ ]:
import os
import glob
test_files = []
test_dir = "/content/test/"
dir_png = os.path.join(test_dir,"*.png")
dir_jpg = os.path.join(test_dir,"*.jpg")
dir_jpeg = os.path.join(test_dir,"*.jpeg")
png_files = glob.glob(dir_png)
jpg_files = glob.glob(dir_jpg)
jpeg_files = glob.glob(dir_jpeg)
test_files.extend(png_files)
test_files.extend(jpg_files)
test_files.extend(jpeg_files)
test_files.sort()

In [ ]:
len(test_files)

In [ ]:
dic = {}
for j in range(len(test_files)):
  b = test_files[j].split("/")[-1]
  dic[b] = test_files[j]

In [ ]:
with open('/content/test.txt') as f:
    lines = f.readlines()

In [ ]:
lines[0].split(" ")

In [ ]:
covid_test = []
Noncovid_test = []
for i in range(len(lines)):
  a = lines[i].split(" ")
  if a[2] == "positive": 
    covid_test.append(dic[a[1]])
  else:
    try:
      Noncovid_test.append(dic[a[1]])
    except:
      print(a[1]) 
covid_test.sort()
Noncovid_test.sort()

In [ ]:
len(covid_test)

In [ ]:
test_dic = {}
for f in covid_test:
  test_dic[f] = [1,0]
for f in Noncovid_test:
  test_dic[f] = [0,1]

In [ ]:
import random
l = list(test_dic.items())
random.Random(4).shuffle(l)

In [ ]:
import cv2
import numpy as np
import numpy as np
import keras
from skimage.filters import roberts, sobel, scharr, prewitt
covid_dic_list = {}
data = []
labels = []
for i in range(len(l)):
  file_name,label = l[i]
  img = cv2.imread(file_name)
  img0 = cv2.imread(file_name,0)   
  try:
    img = np.asarray(img)
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    img = img.astype('float32')/255.0
    data.append(img)
    labels.append(label)
  except:
    print(i,file_name)
    print("Not possible")        
test3_data = np.array(data)
print(test3_data.shape)
test3_labels = np.array(labels)
print(test3_labels.shape)  
print('^_^-training data finished-^_^')

In [ ]:
import cv2
import numpy as np
import numpy as np
import keras
from skimage.filters import roberts, sobel, scharr, prewitt

covid_dic_list = {}
data = []
labels = []
for i in range(len(l_val)):
  file_name,label = l_val[i]
  img = cv2.imread(file_name)
  img0 = cv2.imread(file_name,0)     
  try:
    img = np.asarray(img)
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    img = img.astype('float32')/255.0
    data.append(img)
    labels.append(label)
  except:
    print(i,file_name)
    print("Not possible")  
       
val3_data = np.array(data)
print(val3_data.shape)
val3_labels = np.array(labels)
print(val3_labels.shape)  
print('^_^-validation data finished-^_^')

In [ ]:
inception_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet',classes = 2)

In [ ]:
inception_model.trainable = True

In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))
x = inception_model(inputs)
input_ = tf.expand_dims(x,axis = 1)
nb_chan = 512
ratio = 16
x3 = ConvLSTM2D(filters=512, kernel_size=(1,1),padding = "same")(input_) 
x3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001)(x3)
x3 = Activation('relu')(x3)

y = tf.keras.layers.GlobalAveragePooling2D()(x3)
y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
y_3 = tf.keras.layers.Multiply()([x3, y])

ratio = 16
flat = Flatten()(y_3)

dense_1 = Dense(4096,activation = 'relu')(flat)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
prediction = Dense(2,activation = 'softmax')(dense_2)
inception_pred = Model(inputs = inputs,outputs = prediction)

In [ ]:
inception_pred.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

In [ ]:
inception_pred.fit(l_dataset,epochs = 20)

In [ ]:
inception_pred.evaluate(test3_data,test3_labels)

In [ ]:
inception_pred.load_weights("fintune-10-covidx-inception_combined.h5")

In [ ]:
vgg_model = tf.keras.applications.VGG19(include_top=False,weights='imagenet',classes = 2)

In [ ]:
vgg_model.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))
x = vgg_model(inputs)
input_ = tf.expand_dims(x,axis = 1)
nb_chan = 512
ratio = 16
x3 = ConvLSTM2D(filters=512, kernel_size=(1,1),padding = "same")(input_) 
x3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001)(x3)
x3 = Activation('relu')(x3)

y = tf.keras.layers.GlobalAveragePooling2D()(x3)
y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
y_3 = tf.keras.layers.Multiply()([x3, y])

ratio = 16
flat = Flatten()(y_3)

dense_1 = Dense(4096,activation = 'relu')(flat)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
prediction = Dense(2,activation = 'softmax')(dense_2)
vgg_pred = Model(inputs = inputs,outputs = prediction)

In [ ]:
vgg_pred.summary()
vgg_pred.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

In [ ]:
vgg_pred.fit(l_dataset,epochs = 20)

In [ ]:
vgg_pred.evaluate(test3_data,test3_labels)

13/13 [==============================] - 6s 205ms/step - loss: 14.4267 - accuracy: 0.8350


[14.426671981811523, 0.8349999785423279]

In [ ]:
vgg_pred.load_weights("covidx-vgg_combined.h5")

In [ ]:
mobile_model = tf.keras.applications.MobileNet(input_shape=(224,224,3),include_top=False,weights='imagenet',classes = 2)

In [ ]:
mobile_model.summary()

In [ ]:
mobile_model.trainable = True

In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))
#x = inputs
x = mobile_model(inputs)
input_ = tf.expand_dims(x,axis = 1)
nb_chan = 512
ratio = 16
x3 = ConvLSTM2D(filters=512, kernel_size=(1,1),padding = "same")(input_) 
x3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001)(x3)
x3 = Activation('relu')(x3)

y = tf.keras.layers.GlobalAveragePooling2D()(x3)
y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
y_3 = tf.keras.layers.Multiply()([x3, y])

ratio = 16
flat = Flatten()(y_3)
dense_1 = Dense(4096,activation = 'relu')(flat)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
prediction = Dense(2,activation = 'softmax')(dense_2)
mobile_pred = Model(inputs = inputs,outputs = prediction)

In [ ]:
mobile_pred.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

In [ ]:
mobile_pred.fit(l_dataset,epochs = 10)

In [ ]:
mobile_pred.evaluate(test3_data,test3_labels)

In [ ]:
mobile_pred.load_weights("fintune-covidx-10-mobile_combined.h5")

In [ ]:
test3 = vgg_pred.predict(test3_data)
test4 = mobile_pred.predict(test3_data)
test5 = inception_pred.predict(test3_data)

In [ ]:
test = new_pred.predict(test3_data)
new_test = np.argmax(test,axis = 1)
labels = np.argmax(test3_labels,axis = 1)

In [ ]:
new_test3 = np.argmax(test3,axis = 1)
new_test4 = np.argmax(test4,axis = 1)
new_test5 = np.argmax(test5,axis = 1)
labels = np.argmax(test3_labels,axis = 1)

In [ ]:
val3 = vgg_pred.predict(val3_data)
val4 = mobile_pred.predict(val3_data)
val5 = inception_pred.predict(val3_data)

In [ ]:
new_val3 = np.argmax(val3,axis = 1)
new_val4 = np.argmax(val4,axis = 1)
new_val5 = np.argmax(val5,axis = 1)
val_labels = np.argmax(val3_labels,axis = 1)

In [ ]:
def sugeno(solution,pred1,pred2,pred3, labels):
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      if lam[0] >= 0:
          lam = np.asarray(lam[0])
      elif lam[1] >= 0:
          lam = np.asarray(lam[1])
      elif lam[2] >= 0:
          lam = np.asarray(lam[2])
    
    Ypred_fuzzy = np.zeros(shape = pred1.shape, dtype = float)
    for sample in range(0,pred1.shape[0]):
        for classes in range(0,2):
            scores = np.array([pred1[sample][classes],pred2[sample][classes],pred3[sample][classes]])
            permutedidx = np.flip(np.argsort(scores))
            scoreslambda = scores[permutedidx]
            fmlambda = fuzzymeasures[permutedidx]
            ge_prev = fmlambda[0]
            fuzzyprediction = min((scoreslambda[0], fmlambda[0]))
            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = max((fuzzyprediction,min((scoreslambda[i],ge_curr))))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((2))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(labels,pred_label)
    #print(acc)
    return -acc,ypred_fuzzy

In [ ]:
from sklearn.metrics import accuracy_score
from sympy.solvers import solve
from sympy import Symbol
import numpy as np
# The fuzzy measure values selected by experimentally tuning on the validation set is [1.17979882,1.73704556,2.14123453]
acc, ypred = sugeno([1.17979882,1.73704556,2.14123453],test3,test4,test5,test3_labels)

In [ ]:
acc